# NRS IFU plotting notebook

Author: Jonathan Aguilar (jaguilar@stsci.edu)

This notebook visualizes source placement for NIRSpec IFU observations, for a given orientation of the telescope. It shows the footprint of individual NIRSpec slices within a given channel. For a more general introduction to WHIPPOT, see [WHIPPOT_tutorial](WHIPPOT_tutorial).

WHIPPOT was originally designed for observations that use target acquisition on one star and then manually command the slew to another. Generally, users planning without TA or with self-TA should simply provide the same coordinates for their ACQ target as for their SCI target. 

The user provides the RA and Dec coordinates for the target acquisition (ACQ) star and science (SCI) star, and a telescope roll angle. These tools will then compute a) the "Special Requirement Offset" X and Y values needed to place the SCI star at the reference position (if performing manual offset TA), and b) the final coordinates of the SCI and ACQ stars. Coordinates are given in IDL coordinates, in units of arcsec (https://jwst-docs.stsci.edu/jwst-observatory-hardware/jwst-target-observability-and-observatory-coordinate-system/jwst-instrument-ideal-coordinate-systems#gsc.tab=0).

The user can also provide a list of extra targets in the field in the format specified below, and their IDL coordinates will also be computed. As such, this notebook is useful not just for planning target acquisition, but also for predicting to high precision the final positions of astronomical bodies in the field of view.

## Setup ##

  1. Navigate in a terminal to the directory where you have downloaded the module. This should contain the `setup.py` file.
  2. Activate your desired python environment.
  3. Run `pip install .` (or `pip install -e.` for developing)
  4. Copy this notebook to your working directory, and run your jupyter kernel in the appropriate environment.

In [ ]:
# for development purposes
%load_ext autoreload
%autoreload 2

In [ ]:
# for your system, choose an appropriate plotting backend to make pop-up window plots
# (currently, inline plots show up in the log but not directly in the notebook).
# %matplotlib throws an error if you run it first, so do inline first.
%matplotlib inline
%matplotlib osx

In [ ]:
from whippot import whippot_tools

## An example system

Let's say the star HIP 997700 has a companion with a known offset. Let's make sure the diffraction spikes don't overlap.

In [ ]:
# HIP 99770: 20 14 32.0323540824 +36 48 22.700909124
hip99770 = whippot_tools.SkyCoord("20 14 32.0323540824 +36 48 22.700909124", frame='icrs', unit=("hourangle","degree"))
# let's say the companion has an offset of dRA = 0.5 arcsec, dDec = 0.5 arcsec
companion = hip99770.spherical_offsets_by(0.5*units.arcsec, 0.5*units.arcsec)
sources = {
    'SCI': hip99770,
    'b': companion,
}

In [ ]:
from astropy import units
initial_values={
    'instr': 'nirspec',
    'sci_aper': 'nrs_full_ifu', 
    'pa': 305.,
    'sci_ra': sources['SCI'].ra.deg, 'sci_dec': sources['SCI'].dec.deg,
    'show_diffraction_spikes': True,
}
# add a multi-line string of the other stars, copied from the cell above
initial_values['other_stars'] = "\n".join(f"{k}: ({v.ra.deg}, {v.dec.deg})" for k, v in sources.items() if k not in ['SCI'])

## Use the NIRSpec IFU-specific ComputePositions class

The ComputePositions class in `nrs_ifu_tools.py` is a subclass of the general `ComputePositions` class that adds NRS IFU-specific overlays to the plots.

In [ ]:
from whippot.modes import nrs_ifu_tools
cp1 = nrs_ifu_tools.ComputePositions(initial_values=initial_values)
cp1.ui

In [ ]:
fig = cp1.plot_scene()